# Instalaciones

In [ ]:
!pip install transformers==4.33

In [ ]:
!pip install --upgrade datasets torch pandas keras_preprocessing

  Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia

# Prueba Bert

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

In [ ]:
# Carga el archivo Excel
df = pd.read_excel('U20190876_corpus.xlsx')

# Supongamos que los cuentos están en una columna llamada 'cuento'
textos = df['Body'].tolist()

# Crear un dataset de Hugging Face
dataset = Dataset.from_dict({'text': textos})

In [ ]:
# Cargar el tokenizador
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Definir la función de tokenización y creación de etiquetas
def tokenize_and_mask(examples):
    tokenized_inputs = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)
    tokenized_inputs["labels"] = labels["input_ids"]
    return tokenized_inputs

# Aplicar la función de tokenización y enmascaramiento
tokenized_datasets = dataset.map(tokenize_and_mask, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
# Usar DataCollator para MLM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

In [ ]:
# Cargar el modelo preentrenado
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
model =

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

loading weights file model.safetensors f

In [ ]:
# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="no",
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Configurar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
# Entrenar el modelo
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text. If text are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9
  Number of trainable parameters = 177974523


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=9, training_loss=4.355426364474827, metrics={'train_runtime': 156.8258, 'train_samples_per_second': 0.325, 'train_steps_per_second': 0.057, 'total_flos': 3359348294400.0, 'train_loss': 4.355426364474827, 'epoch': 3.0})

In [ ]:
model.save_pretrained('./shipibo_bert')
tokenizer.save_pretrained('./shipibo_bert')

Configuration saved in ./shipibo_bert/config.json
Model weights saved in ./shipibo_bert/pytorch_model.bin
tokenizer config file saved in ./shipibo_bert/tokenizer_config.json
Special tokens file saved in ./shipibo_bert/special_tokens_map.json


('./shipibo_bert/tokenizer_config.json',
 './shipibo_bert/special_tokens_map.json',
 './shipibo_bert/vocab.txt',
 './shipibo_bert/added_tokens.json')

In [ ]:
from transformers import pipeline

# Cargar el modelo ajustado
model_path = './shipibo_bert'
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Generar un cuento
output = generator("Wetsa neteronki", max_length=100, num_return_sequences=1)
print(output[0]['generated_text'])

The model 'BertForMaskedLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GPT2LMHeadModel', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RoCBertForCausalLM', 'RoFormerForCausalLM', 'Speech2Text2ForCausalLM', 'TransfoXLLMHeadModel', 'TrOCRForCausalLM', 'XGLMForCausalLM', 'XLMWithLMHeadModel', 'XLMProphetNetForCausalLM', 'XLM

Wetsa neteronki netronsa ja kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat


# BertForMaskedLM

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

# Carga el archivo Excel
df = pd.read_excel('U20190876_corpus.xlsx')

# Supongamos que los cuentos están en una columna llamada 'cuento'
textos = df['Body'].tolist()

# Crear un dataset de Hugging Face
dataset = Dataset.from_dict({'text': textos})

# Cargar el tokenizador
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Definir la función de tokenización y creación de etiquetas
def tokenize_and_mask(examples):
    tokenized_inputs = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)
    tokenized_inputs["labels"] = labels["input_ids"]
    return tokenized_inputs

# Aplicar la función de tokenización y enmascaramiento
tokenized_datasets = dataset.map(tokenize_and_mask, batched=True)

# Usar DataCollator para MLM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# Cargar el modelo preentrenado
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="no",
)

# Configurar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Entrenar el modelo
trainer.train()

# Guardar el modelo ajustado
model.save_pretrained('./shipibo_bert')
tokenizer.save_pretrained('./shipibo_bert')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in ./shipibo_bert/config.json
Model weights saved in ./shipibo_bert/pytorch_model.bin
tokenizer config file saved in ./shipibo_bert/tokenizer_config.json
Special tokens file saved in ./shipibo_bert/special_tokens_map.json


('./shipibo_bert/tokenizer_config.json',
 './shipibo_bert/special_tokens_map.json',
 './shipibo_bert/vocab.txt',
 './shipibo_bert/added_tokens.json')

In [ ]:
from transformers import pipeline

# Cargar el modelo ajustado
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Generar un cuento
output = generator("Wetsa neteronki", max_length=100, num_return_sequences=1)
print(output[0]['generated_text'])

The model 'BertForMaskedLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GPT2LMHeadModel', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RoCBertForCausalLM', 'RoFormerForCausalLM', 'Speech2Text2ForCausalLM', 'TransfoXLLMHeadModel', 'TrOCRForCausalLM', 'XGLMForCausalLM', 'XLMWithLMHeadModel', 'XLMProphetNetForCausalLM', 'XLM

Wetsa neteronki netronsa ja kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat kat


# GPT-2

In [ ]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
from datasets import Dataset

# Carga el archivo Excel
df = pd.read_csv('corpus.csv')

# Supongamos que los cuentos están en una columna llamada 'Body'
textos = df['Body'].tolist()

# Guardar los textos en un archivo para usar con TextDataset
with open('cuentos_shipibo.txt', 'w', encoding='utf-8') as f:
    for cuento in textos:
        f.write(cuento + '\n')


ModuleNotFoundError: No module named 'datasets'

In [ ]:
# Cargar el tokenizador y el modelo preentrenado GPT-2
tokenizer_GPT2 = GPT2Tokenizer.from_pretrained('gpt2')
model_GPT2 = GPT2LMHeadModel.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Crear el dataset de entrenamiento
train_dataset = TextDataset(
    tokenizer=tokenizer_GPT2,
    file_path='cuentos_shipibo.txt',
    block_size=128,
)

# Crear el data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer_GPT2,
    mlm=False,
)

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    evaluation_strategy="no",
)

# Crear el Trainer
trainer = Trainer(
    model=model_GPT2,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# Entrenar el modelo
trainer.train()

# Guardar el modelo ajustado
model_GPT2.save_pretrained('./shipibo_gpt2')
tokenizer_GPT2.save_pretrained('./shipibo_gpt2')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 295
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 111
  Number of trainable parameters = 124439808


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in ./shipibo_gpt2/config.json
Model weights saved in ./shipibo_gpt2/pytorch_model.bin
tokenizer config file saved in ./shipibo_gpt2/tokenizer_config.json
Special tokens file saved in ./shipibo_gpt2/special_tokens_map.json


('./shipibo_gpt2/tokenizer_config.json',
 './shipibo_gpt2/special_tokens_map.json',
 './shipibo_gpt2/vocab.json',
 './shipibo_gpt2/merges.txt',
 './shipibo_gpt2/added_tokens.json')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Generar un cuento
from transformers import pipeline

# Cargar el modelo ajustado
generator = pipeline('text-generation', model=model_GPT2, tokenizer=tokenizer_GPT2, device=device)

# Generar un cuento
output = generator("Wetsa neteronki", max_length=100, num_return_sequences=1)
print(output[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Wetsa neteronki ronki, joni iketian, ma ronki jakin mokó iká iki. Jatian jawen matia iki, piká iki namaká iki, makenobonki wetsa iki. Jaskatax ja namanakin iká iki. Ja  kana ichí benki, ea imi, moa bai.


# GPT2 - Test2

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling, AutoTokenizer

# Carga el archivo Excel
df = pd.read_csv('corpus.csv')

# Supongamos que los cuentos están en una columna llamada 'Body'
textos = df['Body'].tolist()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:
# Dividir el corpus en entrenamiento y prueba (80% entrenamiento, 20% prueba)
train_texts, test_texts = train_test_split(textos, test_size=0.2, random_state=42)

In [ ]:
# Guardar los textos en archivos para usar con TextDataset
with open('train_cuentos_shipibo.txt', 'w', encoding='utf-8') as f:
    for cuento in train_texts:
        f.write(cuento + '\n')

with open('test_cuentos_shipibo.txt', 'w', encoding='utf-8') as f:
    for cuento in test_texts:
        f.write(cuento + '\n')

In [ ]:
# Cargar el tokenizador y el modelo preentrenado GPT-2
tokenizer_t2 = AutoTokenizer.from_pretrained("facebook/mms-tts-shp")
model_t2 = GPT2LMHeadModel.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Crear el dataset de entrenamiento
train_dataset = TextDataset(
    tokenizer=tokenizer_t2,
    file_path='train_cuentos_shipibo.txt',
    block_size=128,
)

# Crear el dataset de prueba
test_dataset = TextDataset(
    tokenizer=tokenizer_t2,
    file_path='test_cuentos_shipibo.txt',
    block_size=128,
)

# Crear el data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer_t2,
    mlm=False,
)

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=16,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    evaluation_strategy="epoch",  # Evaluar al final de cada época
)

# Crear el Trainer
trainer = Trainer(
    model=model_t2,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# Entrenar el modelo
trainer.train()

# Guardar el modelo ajustado
model_t2.save_pretrained('./shipibo_gpt2v2')
tokenizer_t2.save_pretrained('./shipibo_gpt2v2')

Epoch,Training Loss,Validation Loss
1,No log,1.774351
2,No log,1.601550
3,No log,1.547320
4,No log,1.492881
5,1.731900,1.453004
6,1.731900,1.437947
7,1.731900,1.428836
8,1.731900,1.423695
9,1.377100,1.420161
10,1.377100,1.412389


('./shipibo_gpt2v2/tokenizer_config.json',
 './shipibo_gpt2v2/special_tokens_map.json',
 './shipibo_gpt2v2/vocab.json',
 './shipibo_gpt2v2/added_tokens.json')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Generar un cuento
from transformers import pipeline

# Cargar el modelo ajustado
generator = pipeline('text-generation', model=model_t2, tokenizer=tokenizer_t2, device=device)

# Generar un cuento
output = generator("Wetsa neteronki", max_length=200, num_return_sequences=1)
print(output[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Wetsa neteronki oxokárpeinreeeioiko eiimibpinmiboxoo iikiitipokotieteneeii enmiboo enmiisoxoiwestoiox


# LSTM

In [ ]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import re

In [ ]:
# Carga el archivo Excel
df = pd.read_csv('corpus.csv')

# Supongamos que los cuentos están en una columna llamada 'Body'
textos = df['Body'].tolist()

# Guardar los textos en un archivo para usar con TextDataset
with open('cuentos_shipibo.txt', 'w', encoding='utf-8') as f:
    for cuento in textos:
        f.write(cuento + '\n')

In [ ]:
file = open("cuentos_shipibo.txt", "r", encoding="utf-8")
text = file.read()
file.close()

In [ ]:
def text_cleaner(text):
    # Convertimos a minúsculas
    new_text = text.lower()
    # Reemplazamos las 's por espacio cadena vacía
    new_text = re.sub(r"'s\b","", new_text)
    # Reemplazamos todos los caracteres diferentes a [a-z] o [A-Z] por espacio en blanco
    new_text = re.sub("[^a-zA-Z]", " ", new_text)
    # Reemplazamos 2 o más espacios en blanco por uno solo
    new_text = re.sub("\s+", " ", new_text)
    # Removemos caracter vacios al inicio o final
    new_text = new_text.strip()
    return new_text

# Preprocesamos
text_cleaned = text_cleaner(text)
text_cleaned

'ja rekena inkan jane ronki ipaonike yo shiko inka ja ronki ipaonike ikonbires yoashi jaw ki yok kana ronki ipaonike ikonbiressiki jatoki yoashii jaskara iken ronki apaokanike yo shiko inka akin ja yo shiko inkan ronki jato ikina ipaoni ke xetebo jonibo ja inka ronki ipaonike wain teetai joni jat bi jaw ki ban ya ato xeki atsa paranta xawi kari poa kankan jat bi jaw ki banai joni jaskarabiribi ronki ipaonike jawen tee jainoaxki ipaonike jat bi yobin banayaribi nato kaimito naranxa xenan bimpish rima rimon tap riba xawen meyari isonxoma tansharina ja jaw kibo banai ronki ipaonike jainxonribiki jat bi yoinnabo in ya inka ronki ik tiai nato waka kab yo karaniro chibo kochi och ti atapa nonon korokoro ja jaw kibo in ya ronki ik tiai ja yo shiko inka jatianki jat bi jaw kiati jaw qkiboki ik tiai jaibabicho jatian ronki apaokan ke jawen xetebo jonibaonki jaw kiati yok kin ja jaw kiati yok na ronki ipaonike jatokiki ikonbiressi yoashii wetsatianki menikinbi jato ich tamashoko menii ichaxon ja

In [ ]:
seq_length = 10
def create_seq(text):
    sequences = []
    for i in range(seq_length, len(text)):
        start = i - seq_length
        end = i + 1
        seq = text[start:end]
        sequences.append(seq)

    print("Total de secuencias: {}".format(len(sequences)))

    return sequences

# Creamos las secuencias
sequences = create_seq(text_cleaned)

Total de secuencias: 78252


In [ ]:
chars = sorted(list(set(text_cleaned)))
print(chars)

[' ', 'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
len(chars)

26

In [ ]:
mapping = dict((c, i) for i, c in enumerate(chars))
mapping

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25}

In [ ]:
def encode_seq(seq):
    sequences = []
    for line in seq:
        encoded_seq = [mapping[char] for char in line]
        sequences.append(encoded_seq)

    return sequences

sequences = encode_seq(sequences)

In [ ]:
# Tamaño del vocabulario
vocab_size = len(mapping)
sequences = np.array(sequences)

# Creamos nuestras variables independientes (X) y dependientes (y), donde X estará compuesto por las 10 primeros caracteres y Y por el último caracter.
X = sequences[:,:-1]
y = sequences[:,-1]

# Obtenemos los one-hot enconding
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size)

# Finalmente separamos el conjunto de datos en dos: entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, random_state=42)

print("Dimensiones del conjunto de datos X de entrenamiento: {}".format(X_train.shape))
print("Dimensiones del conjunto de datos X de validación: {}".format(X_val.shape))

print("Dimensiones del conjunto de datos 'y' de entrenamiento: {}".format(y_train.shape))
print("Dimensiones del conjunto de datos 'y' de validación: {}".format(y_val.shape))

Dimensiones del conjunto de datos X de entrenamiento: (74339, 10, 26)
Dimensiones del conjunto de datos X de validación: (3913, 10, 26)
Dimensiones del conjunto de datos 'y' de entrenamiento: (74339, 26)
Dimensiones del conjunto de datos 'y' de validación: (3913, 26)


In [ ]:
model = Sequential()
model.add(LSTM(75, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(vocab_size, activation="softmax"))
print(model.summary())

# Compilamos el modelo
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")
# Entrenamos el modelo
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 75)                30600     
                                                                 
 dense (Dense)               (None, 26)                1976      
                                                                 
Total params: 32576 (127.25 KB)
Trainable params: 32576 (127.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/20
2324/2324 [==============================] - 25s 8ms/step - loss: 1.9788 - accuracy: 0.3842 - val_loss: 1.7395 - val_accuracy: 0.4429
Epoch 2/20
2324/2324 [==============================] - 20s 9ms/step - loss: 1.6691 - accuracy: 0.4732 - val_loss: 1.6043 - val_accuracy: 0.4810
Epoch 3/20
2324/2324 [==============================] - 19s 8ms/step - loss: 1.5519 - accuracy: 0.5136 - val_loss: 1.51

In [ ]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    # model: modelo
    # mapping: diccionario de codificación de caracteres
    # seq_lentgh: tamaño de la secuencia
    # seed_text: texto inicial con el cuál generará los siguientes caracteres
    # n_chars: número de caracteres a predecir
    in_text = seed_text

    # Iteramos para cada caracter a predecir
    for _ in range(n_chars):

        # Codificamos cada caracter de entrada
        encoded = [mapping[char] for char in in_text]

        # Truncamos la secuencia a un tamaño fijo
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating="pre")
        encoded = to_categorical(encoded, num_classes=len(mapping))

        # Predecimos un caracter
        predict_x = model.predict(encoded, verbose=0)

        yhat = np.argmax(predict_x, axis=1)
        # Decodificamos el caracter predicho
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # Y lo añadimos al final del texto
        in_text += char
    return in_text

In [ ]:
input_text = "wetsa neteronki"
generate_seq(model, mapping, seq_length, input_text, 30)

'wetsa neteronki ika iki ja nete jatian jatian'